<a href="https://colab.research.google.com/github/Ictuer/airbnb-be/blob/main/examples/notebook/sat/cp_sat_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# cp_sat_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/sat/cp_sat_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/sat/samples/cp_sat_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [2]:
%pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.


Simple solve.

In [19]:
from ortools.sat.python import cp_model
import json

def solve_class_scheduling(sessions, rooms, students, min_students_per_class=3):
    """
    Giải bài toán xếp lớp học sử dụng OR-Tools CP-SAT
    """

    # Khởi tạo model
    model = cp_model.CpModel()

    # ===== TÍNH TOÁN CÁC THÔNG SỐ CƠ BẢN =====

    # Tính số lớp tối đa có thể mở cho mỗi session
    max_classes_per_session = {}
    for session_id, session in sessions.items():
        num_registers = len(session['registers'])
        max_classes_per_session[session_id] = num_registers // min_students_per_class

    print("Số lớp tối đa có thể mở:")
    for session_id, max_classes in max_classes_per_session.items():
        print(f"  {session_id}: {max_classes} lớp (từ {len(sessions[session_id]['registers'])} học viên)")

    # ===== KHỞI TẠO CÁC BIẾN QUYẾT ĐỊNH =====

    # 1. Biến quyết định lớp được mở
    class_opened = {}
    for session_id in sessions:
        class_opened[session_id] = {}
        for class_idx in range(max_classes_per_session[session_id]):
            class_opened[session_id][class_idx] = model.NewBoolVar(
                f'class_opened_{session_id}_{class_idx}'
            )

    # 2. Biến gán phòng cho lớp - chỉ tạo cho phòng cùng facility
    room_assigned = {}
    for session_id in sessions:
        session_facility = sessions[session_id]['facility']
        room_assigned[session_id] = {}
        for class_idx in range(max_classes_per_session[session_id]):
            room_assigned[session_id][class_idx] = {}
            # Chỉ tạo biến cho các phòng cùng facility
            for room_id, room_data in rooms.items():
                if room_data['facility'] == session_facility:
                    room_assigned[session_id][class_idx][room_id] = model.NewBoolVar(
                        f'room_assigned_{session_id}_{class_idx}_{room_id}'
                    )

    # 3. Biến gán học viên vào lớp
    student_in_class = {}
    for session_id in sessions:
        student_in_class[session_id] = {}
        for class_idx in range(max_classes_per_session[session_id]):
            student_in_class[session_id][class_idx] = {}
            for student_id in sessions[session_id]['registers']:
                student_in_class[session_id][class_idx][student_id] = model.NewBoolVar(
                    f'student_in_class_{session_id}_{class_idx}_{student_id}'
                )

    # ===== RÀNG BUỘC =====

    # A. Ràng buộc gán phòng theo facility - đơn giản vì đã lọc từ đầu
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            # Nếu lớp được mở, phải chọn đúng 1 phòng (đã lọc cùng facility)
            available_rooms = list(room_assigned[session_id][class_idx].keys())
            if available_rooms:
                model.Add(
                    sum(room_assigned[session_id][class_idx][room_id]
                        for room_id in available_rooms) == class_opened[session_id][class_idx]
                )
            else:
                # Nếu không có phòng phù hợp, không thể mở lớp
                model.Add(class_opened[session_id][class_idx] == 0)

    # B. Kiểm tra xung đột thời gian phòng - xử lý từng slot riêng biệt
    for room_id in rooms:
        room_data = rooms[room_id]

        # Tạo danh sách các interval cho phòng này
        intervals_for_room = []

        # 1. Thêm lịch bận của phòng
        for busy_slot in room_data['busy']:
            busy_interval = model.NewIntervalVar(
                busy_slot['begin'],
                busy_slot['end'] - busy_slot['begin'],
                busy_slot['end'],
                f'room_busy_{room_id}_{busy_slot["begin"]}_{busy_slot["end"]}'
            )
            intervals_for_room.append(busy_interval)

        # 2. Thêm từng slot của các lớp có thể được gán vào phòng này
        for session_id in sessions:
            session_data = sessions[session_id]
            # Chỉ xét các session có cùng facility
            if session_data['facility'] == room_data['facility']:
                for class_idx in range(max_classes_per_session[session_id]):
                    # Thêm từng slot của lớp này
                    for slot_idx, slot in enumerate(session_data['slots']):
                        # Tạo interval mới có điều kiện cho slot này trong phòng
                        slot_duration = slot['end'] - slot['begin']

                        # Tạo biến boolean kết hợp: lớp mở VÀ phòng được gán
                        class_slot_in_room_condition = model.NewBoolVar(
                            f'condition_{session_id}_{class_idx}_{room_id}_slot_{slot_idx}'
                        )

                        # class_slot_in_room_condition = class_opened AND room_assigned
                        model.AddBoolAnd([
                            class_opened[session_id][class_idx],
                            room_assigned[session_id][class_idx][room_id]
                        ]).OnlyEnforceIf(class_slot_in_room_condition)

                        model.AddBoolOr([
                            class_opened[session_id][class_idx].Not(),
                            room_assigned[session_id][class_idx][room_id].Not()
                        ]).OnlyEnforceIf(class_slot_in_room_condition.Not())

                        class_slot_in_room = model.NewOptionalIntervalVar(
                            slot['begin'],
                            slot_duration,
                            slot['end'],
                            class_slot_in_room_condition,
                            f'room_{room_id}_class_{session_id}_{class_idx}_slot_{slot_idx}'
                        )
                        intervals_for_room.append(class_slot_in_room)

        # Đảm bảo không overlap
        if len(intervals_for_room) > 1:
            model.AddNoOverlap(intervals_for_room)

    # C. Kiểm tra xung đột thời gian học viên (chỉ giữa các môn khác nhau)
    for student_id in students:
        # Nhóm các session theo môn học mà học viên đăng ký
        sessions_by_subject = {}
        for session_id in sessions:
            if student_id in sessions[session_id]['registers']:
                subject = sessions[session_id]['subject']
                if subject not in sessions_by_subject:
                    sessions_by_subject[subject] = []
                sessions_by_subject[subject].append(session_id)

        # Tạo interval cho mỗi slot của mỗi môn học
        all_subject_intervals = []
        subjects_list = list(sessions_by_subject.keys())

        # Đối với mỗi cặp môn khác nhau, không được học cùng lúc
        for i in range(len(subjects_list)):
            for j in range(i + 1, len(subjects_list)):
                subject1, subject2 = subjects_list[i], subjects_list[j]

                intervals_to_check = []

                # Thêm tất cả intervals của subject1
                if subject1 in student_subject_intervals.get(student_id, {}):
                    intervals_to_check.extend(student_subject_intervals[student_id][subject1])

                # Thêm tất cả intervals của subject2
                if subject2 in student_subject_intervals.get(student_id, {}):
                    intervals_to_check.extend(student_subject_intervals[student_id][subject2])

                if intervals_to_check:
                    model.AddNoOverlap(intervals_to_check)

    # D. Ràng buộc số học viên tối thiểu cho lớp
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            students_in_this_class = [
                student_in_class[session_id][class_idx][student_id]
                for student_id in sessions[session_id]['registers']
            ]

            if students_in_this_class:
                student_count = sum(students_in_this_class)

                # Lớp chỉ mở nếu có đủ số học viên tối thiểu
                model.Add(student_count >= min_students_per_class).OnlyEnforceIf(
                    class_opened[session_id][class_idx]
                )

                # Nếu lớp không mở thì không có học viên nào
                model.Add(student_count == 0).OnlyEnforceIf(
                    class_opened[session_id][class_idx].Not()
                )

    # E. Ràng buộc capacity phòng
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            students_count = sum(
                student_in_class[session_id][class_idx][student_id]
                for student_id in sessions[session_id]['registers']
            )

            # Chỉ kiểm tra các phòng đã được tạo biến (cùng facility)
            for room_id in room_assigned[session_id][class_idx]:
                room_capacity = rooms[room_id]['capacity']
                # Nếu lớp được gán vào phòng, số học viên không vượt capacity
                model.Add(students_count <= room_capacity).OnlyEnforceIf(
                    room_assigned[session_id][class_idx][room_id]
                )

    # F. Mỗi học viên chỉ được xếp vào tối đa 1 lớp của mỗi session
    for session_id in sessions:
        for student_id in sessions[session_id]['registers']:
            classes_for_student = [
                student_in_class[session_id][class_idx][student_id]
                for class_idx in range(max_classes_per_session[session_id])
            ]
            if classes_for_student:
                model.Add(sum(classes_for_student) <= 1)

    # ===== HÀM MỤC TIÊU =====
    objective_terms = []

    # Trọng số cao nhất (10000): Số lượng học viên được học
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            for student_id in sessions[session_id]['registers']:
                objective_terms.append(
                    student_in_class[session_id][class_idx][student_id] * 10000
                )

    # Trọng số thứ 2 (-100): Giảm số lớp mở
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            objective_terms.append(class_opened[session_id][class_idx] * (-100))

    # Trọng số thứ 3 (5): Ưu tiên học viên đăng ký ít session
    for student_id, student_data in students.items():
        registered_sessions = max(student_data.get('registered_sessions', 1), 1)
        priority_weight = 5.0 / registered_sessions

        for session_id in sessions:
            if student_id in sessions[session_id]['registers']:
                for class_idx in range(max_classes_per_session[session_id]):
                    objective_terms.append(
                        student_in_class[session_id][class_idx][student_id] * priority_weight
                    )

    # Trọng số thứ 4 (1): Ưu tiên học viên bị hụt môn
    for student_id, student_data in students.items():
        for session_id in sessions:
            subject = sessions[session_id]['subject']
            if student_id in sessions[session_id]['registers']:
                missed_count = student_data.get('missed', {}).get(subject, 0)
                if missed_count > 0:
                    for class_idx in range(max_classes_per_session[session_id]):
                        objective_terms.append(
                            student_in_class[session_id][class_idx][student_id] * missed_count
                        )

    if objective_terms:
        model.Maximize(sum(objective_terms))

    # ===== DEBUG: KIỂM TRA CÁC BIẾN VÀ RÀNG BUỘC =====
    print("\n=== DEBUG: TỔNG QUAN MODEL ===")
    total_vars = 0
    total_constraints = 0

    # Đếm biến
    for session_id in sessions:
        for class_idx in range(max_classes_per_session[session_id]):
            total_vars += 1  # class_opened
            total_vars += len(rooms)  # room_assigned
            total_vars += len(sessions[session_id]['registers'])  # student_in_class

    print(f"Tổng số biến boolean: {total_vars}")
    print(f"Tổng số objective terms: {len(objective_terms) if 'objective_terms' in locals() else 0}")

    # Kiểm tra xem có lớp nào có thể mở không
    for session_id in sessions:
        session_data = sessions[session_id]
        facility = session_data['facility']

        # Tìm phòng phù hợp
        compatible_rooms = [r for r, rd in rooms.items() if rd['facility'] == facility]
        print(f"\nSession {session_id}: {len(compatible_rooms)} phòng phù hợp: {compatible_rooms}")

        if compatible_rooms:
            for room_id in compatible_rooms:
                room_data = rooms[room_id]
                print(f"  Phòng {room_id}:")

                # Kiểm tra từng slot với lịch bận
                for slot_idx, slot in enumerate(session_data['slots']):
                    conflicts = []
                    for busy in room_data['busy']:
                        if not (slot['end'] <= busy['begin'] or slot['begin'] >= busy['end']):
                            conflicts.append(f"busy({busy['begin']}-{busy['end']})")

                    if conflicts:
                        print(f"    Slot {slot_idx} ({slot['begin']}-{slot['end']}): XUNG ĐỘT với {conflicts}")
                    else:
                        print(f"    Slot {slot_idx} ({slot['begin']}-{slot['end']}): OK")

    # ===== GIẢI MODEL =====
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 30.0
    solver.parameters.log_search_progress = True

    print("\n=== BẮT ĐẦU GIẢI MODEL ===")
    status = solver.Solve(model)
    print(f"Trạng thái giải: {solver.StatusName(status)}")
    print(f"Thời gian giải: {solver.WallTime():.2f}s")
    print(f"Objective value: {solver.ObjectiveValue() if status in [cp_model.OPTIMAL, cp_model.FEASIBLE] else 'N/A'}")

    # ===== XỬ LÝ KẾT QUẢ =====
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        result = {
            "opened_classes": [],
            "total_students_served": 0,
            "total_classes_opened": 0,
            "optimization_status": "OPTIMAL" if status == cp_model.OPTIMAL else "FEASIBLE"
        }

        print(f"\n=== KIỂM TRA KẾT QUẢ ===")

        for session_id in sessions:
            print(f"\nSession {session_id}:")
            for class_idx in range(max_classes_per_session[session_id]):
                is_opened = solver.Value(class_opened[session_id][class_idx])
                print(f"  Lớp {class_idx}: {'MỞ' if is_opened else 'ĐÓNG'}")

                if is_opened:
                    # Tìm phòng được gán
                    assigned_room = None
                    for room_id in room_assigned[session_id][class_idx]:
                        if solver.Value(room_assigned[session_id][class_idx][room_id]):
                            assigned_room = room_id
                            break

                    # Tìm học viên trong lớp
                    students_in_class = []
                    for student_id in sessions[session_id]['registers']:
                        if solver.Value(student_in_class[session_id][class_idx][student_id]):
                            students_in_class.append(student_id)

                    print(f"    Phòng: {assigned_room}")
                    print(f"    Học viên ({len(students_in_class)}): {students_in_class}")

                    class_info = {
                        "session": session_id,
                        "class_index": class_idx,
                        "room": assigned_room,
                        "students": students_in_class,
                        "time_slots": sessions[session_id]['slots'],
                        "subject": sessions[session_id]['subject'],
                        "facility": sessions[session_id]['facility']
                    }

                    result["opened_classes"].append(class_info)
                    result["total_students_served"] += len(students_in_class)
                    result["total_classes_opened"] += 1

        return result

    else:
        print(f"Không tìm thấy giải pháp khả thi. Trạng thái: {solver.StatusName(status)}")
        return {
            "opened_classes": [],
            "total_students_served": 0,
            "total_classes_opened": 0,
            "optimization_status": "INFEASIBLE",
            "message": f"Không tìm thấy giải pháp khả thi. Status: {solver.StatusName(status)}"
        }


def check_time_conflicts(sessions, rooms):
    """Kiểm tra xung đột thời gian cơ bản"""
    print("\n=== KIỂM TRA XUNG ĐỘT THỜI GIAN ===")

    for session_id, session in sessions.items():
        facility = session['facility']
        slots = session['slots']

        print(f"\nSession {session_id} ({session['subject']}) tại {facility}:")
        print(f"  Thời gian: {slots}")

        # Tìm phòng phù hợp
        compatible_rooms = []
        for room_id, room in rooms.items():
            if room['facility'] == facility:
                compatible_rooms.append(room_id)
                print(f"  Phòng phù hợp: {room_id}")
                print(f"    Lịch bận: {room['busy']}")
                print(f"    Capacity: {room['capacity']}")

                # Kiểm tra xung đột
                session_start = min(slot['begin'] for slot in slots)
                session_end = max(slot['end'] for slot in slots)

                has_conflict = False
                for busy in room['busy']:
                    if not (session_end <= busy['begin'] or session_start >= busy['end']):
                        has_conflict = True
                        print(f"    ❌ XUNG ĐỘT: Session ({session_start}-{session_end}) vs Busy ({busy['begin']}-{busy['end']})")

                if not has_conflict:
                    print(f"    ✅ KHÔNG XUNG ĐỘT")

        if not compatible_rooms:
            print(f"  ❌ KHÔNG CÓ PHÒNG PHÙ HỢP")


    # ===== TEST VỚI DỮ LIỆU MẪU =====
if __name__ == "__main__":
    # Dữ liệu test với multiple slots để kiểm tra logic mới
    sessions = {
        "SS1": {
            "subject": "Math",
            "facility": "NEU",
            "slots": [{"begin": 4, "end": 6}, {"begin": 7, "end": 9}],  # 2 slots riêng biệt
            "registers": ["a", "b", "c"]
        }
    }

    rooms = {
        "R1": {
            "busy": [{"begin": 1, "end": 3}, {"begin": 10, "end": 15}],
            "facility": "NEU",
            "capacity": 20
        },
        "R2": {
            "busy": [{"begin": 5, "end": 6}],
            "facility": "FTU",
            "capacity": 20
        }
    }

    students = {
        "a": {
            "missed": {"Math": 1},
            "registered_sessions": 1
        },
        "b": {
            "missed": {"Math": 2},
            "registered_sessions": 1
        },
        "c": {
            "missed": {},
            "registered_sessions": 1
        }
    }

    print("=== BẮT ĐẦU GIẢI BÀI TOÁN XẾP LỊCH LỚP HỌC ===")

    # Kiểm tra xung đột thời gian trước
    check_time_conflicts(sessions, rooms)

    result = solve_class_scheduling(sessions, rooms, students, min_students_per_class=3)

    print("\n" + "="*50)
    print("=== KẾT QUẢ CUỐI CÙNG ===")
    print(f"Trạng thái tối ưu hóa: {result['optimization_status']}")
    print(f"Tổng số học viên được phục vụ: {result['total_students_served']}")
    print(f"Tổng số lớp được mở: {result['total_classes_opened']}")

    if result['opened_classes']:
        print("\nChi tiết các lớp được mở:")
        for i, class_info in enumerate(result['opened_classes']):
            print(f"\nLớp {i+1}:")
            print(f"  Session: {class_info['session']}")
            print(f"  Môn học: {class_info['subject']}")
            print(f"  Phòng: {class_info['room']}")
            print(f"  Trung tâm: {class_info['facility']}")
            print(f"  Thời gian: {class_info['time_slots']}")
            print(f"  Học viên ({len(class_info['students'])}): {', '.join(class_info['students'])}")
    else:
        print("\n❌ KHÔNG CÓ LỚP NÀO ĐƯỢC MỞ")
        if 'message' in result:
            print(f"Lý do: {result['message']}")

    # In JSON kết quả
    print(f"\n=== KẾT QUẢ JSON ===")
    print(json.dumps(result, indent=2, ensure_ascii=False))

=== BẮT ĐẦU GIẢI BÀI TOÁN XẾP LỊCH LỚP HỌC ===

=== KIỂM TRA XUNG ĐỘT THỜI GIAN ===

Session SS1 (Math) tại NEU:
  Thời gian: [{'begin': 4, 'end': 6}, {'begin': 7, 'end': 9}]
  Phòng phù hợp: R1
    Lịch bận: [{'begin': 1, 'end': 3}, {'begin': 10, 'end': 15}]
    Capacity: 20
    ✅ KHÔNG XUNG ĐỘT
Số lớp tối đa có thể mở:
  SS1: 1 lớp (từ 3 học viên)

=== DEBUG: TỔNG QUAN MODEL ===
Tổng số biến boolean: 6
Tổng số objective terms: 9

Session SS1: 1 phòng phù hợp: ['R1']
  Phòng R1:
    Slot 0 (4-6): OK
    Slot 1 (7-9): OK

=== BẮT ĐẦU GIẢI MODEL ===
Trạng thái giải: OPTIMAL
Thời gian giải: 0.01s
Objective value: 29918.0

=== KIỂM TRA KẾT QUẢ ===

Session SS1:
  Lớp 0: MỞ
    Phòng: R1
    Học viên (3): ['a', 'b', 'c']

=== KẾT QUẢ CUỐI CÙNG ===
Trạng thái tối ưu hóa: OPTIMAL
Tổng số học viên được phục vụ: 3
Tổng số lớp được mở: 1

Chi tiết các lớp được mở:

Lớp 1:
  Session: SS1
  Môn học: Math
  Phòng: R1
  Trung tâm: NEU
  Thời gian: [{'begin': 4, 'end': 6}, {'begin': 7, 'end': 9}]
  